In [1]:
import numpy as np
import tensorflow as tf


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [18]:

# get data from train_1.csv
tmp = np.genfromtxt("train_3.csv", dtype=float, delimiter=",")

# set training_data quantity and validation data quantity
train_quantity = 750
validation_quantity = 891-train_quantity

# 归一化
tem = tmp[1:,3:]
ave = np.mean(tem,axis = 0)*np.ones((891,5))
ptp = np.ptp(tem,axis = 0)*np.ones((891,5))
tem = np.subtract(tem,ave)
tem = np.true_divide(tem,ptp)
tmp[1:,3:] = tem

#apart train,validation
train_data = tmp[1:train_quantity+1,3:]
validation_data0 = tmp[train_quantity+1:,3:]

train_lable = tmp[1:train_quantity+1,2]
validation_lable0 = tmp[train_quantity+1:,2]

train_lable = train_lable.reshape(train_quantity,1)
validation_lable0 = validation_lable0.reshape(validation_quantity,1)

# define layer function
def add_layer(input,in_size,out_size):
    theta = tf.Variable(tf.random_normal([in_size,out_size]))
    if out_size==1:
        z = tf.matmul(input,theta)
        a = tf.sigmoid(z)
    else:
        z = tf.matmul(input,theta)
        a = tf.nn.relu(z)
        a_0 = tf.ones([train_quantity,1])
        a = tf.concat([a_0,a],1)
    return theta,a,z


# define placeholder for inputs to network
xs = tf.placeholder(tf.float32,[None,5])
ys = tf.placeholder(tf.float32,[None,1])

# add hidden layer
theta_1,l1,z_1 = add_layer(xs,5,8)
# add output layer
theta_2,output,z_2 = add_layer(l1,9,1)

#regularzation
lamda = 0.13
regularzation = (tf.reduce_sum(tf.square(theta_1))+tf.reduce_sum(tf.square(theta_2)))*lamda/600

# the error between prediction and real data
J_theta = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = ys,logits = z_2)) + regularzation

a =  0.5
# learning rate
train_step = tf.train.GradientDescentOptimizer(a).minimize(J_theta)

#prediction
validation_data = tf.constant(validation_data0,tf.float32)
validation_lable = tf.constant(validation_lable0,tf.float32)

a_1 = tf.nn.relu(tf.matmul(validation_data,theta_1))
a_0 = tf.ones([validation_quantity,1])
a_20 = tf.concat([a_0,a_1],1)
a_2 = tf.sigmoid(tf.matmul(a_20,theta_2))
prediction = tf.round(a_2)

validation = tf.subtract(prediction,validation_lable)
validation = tf.abs(validation)
accuracy =tf.reduce_sum(validation) 
accuracy = (validation_quantity-accuracy)/validation_quantity

#initialize
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(20000):
    #training
    sess.run(train_step,feed_dict={xs:train_data,ys:train_lable})
    
    if i%1000 == 0:
        loss = sess.run(J_theta,feed_dict={xs:train_data,ys:train_lable})
        print(i)
        print("J_theta:",loss)
        acc = sess.run(accuracy)
        print(acc)
        print("\n")




0
J_theta: 0.7618145
0.63829786


1000
J_theta: 0.43892616
0.83687943


2000
J_theta: 0.43614057
0.83687943


3000
J_theta: 0.4345359
0.83687943


4000
J_theta: 0.43251747
0.8439716


5000
J_theta: 0.43207785
0.8439716


6000
J_theta: 0.431031
0.858156


7000
J_theta: 0.43057042
0.858156


8000
J_theta: 0.43046874
0.858156


9000
J_theta: 0.43041688
0.858156


10000
J_theta: 0.43038914
0.858156


11000
J_theta: 0.43037358
0.858156


12000
J_theta: 0.43036357
0.858156


13000
J_theta: 0.43035606
0.858156


14000
J_theta: 0.43035153
0.858156


15000
J_theta: 0.43034753
0.858156


16000
J_theta: 0.4303447
0.858156


17000
J_theta: 0.43034214
0.858156


18000
J_theta: 0.43034112
0.858156


19000
J_theta: 0.43033862
0.858156




In [19]:
# import test sets
test = np.genfromtxt("test_3.csv",dtype = float,delimiter=",")

#reshape test sets
tem1 = test[1:,2:]
ave1 = np.mean(tem1,axis = 0)*np.ones((418,5))
ptp1 = np.ptp(tem1,axis = 0)*np.ones((418,5))
tem1 = np.subtract(tem1,ave1)
tem1 = np.true_divide(tem1,ptp1)
test[1:,2:] = tem1
test = test[1:,2:]

# define tensorflow constant test data
test_data = tf.constant(test,tf.float32)

# predict test data
t_1 = tf.nn.relu(tf.matmul(test_data,theta_1))
t_0 = tf.ones([418,1])
t_20 = tf.concat([t_0,t_1],1)
t_2 = tf.sigmoid(tf.matmul(t_20,theta_2))
prediction_1 = tf.round(t_2)
prediction_1 = tf.to_int32(prediction_1)

# return result
result = sess.run(prediction_1)
No = np.arange(892,1310)
No = No.reshape(418,1)
title = np.array(["PassengerId","Survived"])
title = title.reshape(1,2)
result = np.append(No,result,axis=1)
result = np.append(title,result,axis=0)
np.savetxt("result.csv", result, delimiter=",",fmt = "%s")


In [20]:
sess.run(t_2)

array([[0.07828485],
       [0.41773424],
       [0.05639753],
       [0.11642826],
       [0.36232808],
       [0.12406512],
       [0.586018  ],
       [0.17196302],
       [0.7865508 ],
       [0.11544763],
       [0.11619608],
       [0.26725817],
       [0.9605941 ],
       [0.10153506],
       [0.96647125],
       [0.9813595 ],
       [0.11229893],
       [0.3692417 ],
       [0.38864338],
       [0.7710768 ],
       [0.22369397],
       [0.15070243],
       [0.97509944],
       [0.56642   ],
       [0.9337672 ],
       [0.0673252 ],
       [0.9977679 ],
       [0.33428943],
       [0.30294117],
       [0.16674225],
       [0.09558524],
       [0.1932588 ],
       [0.35498488],
       [0.3421932 ],
       [0.4470215 ],
       [0.4304812 ],
       [0.39208424],
       [0.3714012 ],
       [0.11654937],
       [0.11125229],
       [0.08005501],
       [0.3886877 ],
       [0.08873551],
       [0.88742965],
       [0.9698492 ],
       [0.11671314],
       [0.3003355 ],
       [0.092